## Maquinas virtuales

Hasta hace no muchos anios era comun que las empresas tuvieran un servidor por cada aplicativo importante que tubieran y muchas veces tenian directamente servidores independientes. La realidad es que como normalmente el proyecto involucraba comprar un sistema nuevo con requerimientos nuevos, entonces era comun que al encarar un proyecto tambien se involucraba al area de IT para que disponibilice la plataforma donde se iba a correr. Para algunas empresas, lo podian obviar, otras, cuando uno da un servicio y ese servicio esta destinado a una determinada empresa entonces si podia requerir que sea un servidor especifico. Esto le traia problemas al area de IT, lo primero que los data centers son caros en cuanto a lugar, alquilar un rac en un datacenter es caro porque viene con muchas cosas como seguridad, refrigeracion, etc. Toda esa infraesctructra es cara. Con la redundancia ante fallas lo que pasaba era que en primera instancia se caia el servicio, la gente de IT tenia que tener partes de reemplazo, pero si los distintos servidores eran de distintas tecnologias entonces no iba a servir el reemplazo. La otra alternativa era tener servidores completos de reemplazo pero ocuapn lugar y presentan un costo el tenerlos. Un tercer problema era que muchas veces habian recursos propios desaprovechados. Esto es lo que de alguna forma nos va a introducir al concepto de maquinas virtuales. Esto no es algo nuevo, si bien es algo que hizo IBM en la epoca del 60, cayo en desuso y con los problemas vistos anteriores la gente comenzo a ver que no eran tan mala idea. Cuando sacan la IBM S/360 que fue la primera la capacidad de RAM era de 4k. La idea era radicalmente distinta, en vez de correr un unico programa en un unico SO, y ese SO le presentaba las disitnas aplicaciones dispositivos virtuales. La idea es que en vez de un programa llamado SO virtualize a la aplicacion los dispositivos, tenemos otro programa que se llama virtualizador que le virtualiza el hardware a los SO. Entonces, voy a tener por ejemplo un teclado o un mouse y lo voy a distribuir entre esos distintos programas que estan corriendo que cada uno sera llamado maquina virtual. Que cosas deberiamos virtualizar? hay que virtualizar la RAM, tal vez es una de las mas facil de virtualizar, hay que dividirla entre los distintos candidatos, hay que virtualizar los procesadores, cosa que antes estrictamente hacia el SO repartiendo los procesadores entre los threads, ahora cada SO va a tratar de identificiar el procesador que tiene y cuantos cores tiene. Teclado, mouse, placa de video, todo esto tiene que ser hardware virtual. Entonces cada SO va a tener abajo un hardware pensando que es suyo cuando en realidad lo tiene compartido con otros. Este programa que permite crear estas diferentes instancias de hardwaare en cada una de las cuales podre instalar un SO se lo denomina hipervisor. Es como una vista superior al SO. Como queda la estructura ahora? esta virtualizacion del hardware hace que cada uno de los SO vea su propio hardware, ahora teemos la parte fisica y en vez de tenenr el SO sobre la parte fiisca como teniamos antes con sus drivers que manejaban el hardware y a su vez el SO con sus tareas, tenemos sobre el fisico el hipervisor que expone n instancias del hardware llamadas maquinas virtuales y en cada maquina virtual voy a instalar un SO y cada uno de esos SO correran un conjunto de tareas. Es como que agregamos una capa mas. Obviament estas maquinas pueden tener SO distintos. De la misma manera que el SO veia sus propios recuross y protegia a las tareas entre si, el hipervisor hace lo mismo pero entre SOs. El hipervisor hace que cada SO piense que su hardware es exclusivo como el SO hace lo mismo con sus tareas. Y el hipervisor es el que mantiene el orden para que no haya conflicto en el hardware. Esto da un aprovechameinto mucho mas eficiente de los recursos. Cuando un SO no tiene carga o nada que hacer le da control a los otros, y en el mismo hardware puedo correr distintos tipos de aplicaciones. Cuantas mas maquinas correr meor aprovechamiento. 

## Soporte de Hardware

Cada SO piensa que es su hardware, que nadie mas esta compartiendo nada con el, entocnes pretende usarlo de forma directa. De la misma forma que antes teniamos un task swithc cuando habia que sacarle a una tarea el procesador para darselo a otra, lo mismo va a tener que hacer el hipervisor, sacarle el procesador a uno para darselo a otro. Cambiar de una maquina a otra involucraria todo un tema en cuanto a segmentos, por otro lado como hago para que la maquina no use memoria que le dio a otra, se necesita soporte de hardware. Obviamente para no tener que estar entendiendo cada instruccion que cada SO quiere ejecutar, volvemos a crear hardware para detectar esas cosas. Ya hace tiempo que los procesadores tienen hardware especifico para las maquinas virtuales, estan reservados para un hipervisor que pueda controlar los distintos SOs. El hipervisor pasa a correr en ring 0, los SOs en ring 1 y las tareas en ring 3. Los fabricantes hace rato incluyeron diferentes tecnologias en los procesadores. Para que sirven estas nueva instrucciones? minuto 25.
